In [ ]:
import os
import cv2

# 비디오 파일을 처리할 함수
def extract_video_segment(video_path, groundtruth_file, output_folder, video_number, num_frames=30):
    print(f"Trying to open video file: {video_path}")
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    # 비디오 정보 얻기
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video opened: {video_path}, FPS: {fps}, Total Frames: {total_frames}")

    # Groundtruth 파일에서 넘어짐 시작과 끝 프레임 읽기
    try:
        with open(groundtruth_file, 'r', encoding='ISO-8859-1') as file:
            lines = file.readlines()
            start_frame = int(lines[0].strip())  # 시작 프레임
            end_frame = int(lines[1].strip())    # 끝 프레임
    except Exception as e:
        print(f"Error reading groundtruth file {groundtruth_file}: {e}")
        return

    # 구간 설정
    segment_start = max(0, start_frame - 15)
    segment_end = min(total_frames, segment_start + num_frames)

    print(f"Processing video segment from frame {segment_start} to {segment_end}")

    # 비디오 쓰기 객체 생성 (MP4 형식)
    video_name = f"video ({video_number})"  # 기존 번호 그대로 사용
    os.makedirs(output_folder, exist_ok=True)
    
    # 기존 파일명 그대로 저장
    output_video_path = os.path.join(output_folder, f"{video_name}.mp4")
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # 비디오 구간 추출
    cap.set(cv2.CAP_PROP_POS_FRAMES, segment_start)
    for i in range(segment_start, segment_end):
        ret, frame = cap.read()
        if not ret:
            print(f"Error reading frame {i}")
            break
        out.write(frame)
    
    cap.release()
    out.release()
    print(f"Video segment saved to {output_video_path}.")

# 장소별로 비디오와 어노테이션 파일을 처리하는 함수
def process_location_videos(annotation_folder, video_folder, location_name, output_folder):
    print(f"Processing location: {location_name}")
    location_output_folder = os.path.join(output_folder, location_name)
    os.makedirs(location_output_folder, exist_ok=True)

    # Annotation 파일 처리
    annotation_files = [f for f in os.listdir(annotation_folder) if f.endswith('.txt')]
    print(f"Found {len(annotation_files)} annotation files in {annotation_folder}")

    for annotation_file in annotation_files:
        # 비디오 이름을 (i) 형식에 맞게 처리
        video_name = annotation_file.split(' (')[0]  # 괄호 및 숫자 제거
        # 여러 개의 비디오 파일을 처리하도록 각 비디오 이름에 번호 추가
        for i in range(1, 100):  # 예시로 번호 1~99까지 처리 (이 범위는 데이터에 맞게 조정)
            video_file_name = f"{video_name} ({i}).avi"
            video_path = os.path.join(video_folder, video_file_name)
            groundtruth_file = os.path.join(annotation_folder, f"{video_name} ({i}).txt")
            
            # 비디오 파일과 어노테이션 파일이 존재하면 처리
            if os.path.exists(video_path) and os.path.exists(groundtruth_file):
                print(f"Processing video: {video_path}")
                # 비디오 구간 추출
                extract_video_segment(video_path, groundtruth_file, location_output_folder, i)
            else:
                print(f"Video or groundtruth file not found: {video_path} or {groundtruth_file}")

# FallDataset 폴더 내 모든 장소 폴더 탐색
def process_fall_dataset(fall_dataset_folder, output_folder):
    print(f"Starting to process FallDataset in {fall_dataset_folder}")
    
    # FallDataset 폴더가 제대로 있는지 확인
    if not os.path.exists(fall_dataset_folder):
        print(f"Error: {fall_dataset_folder} does not exist.")
        return

    # output_video_segments 폴더 절대 경로로 설정
    output_folder = os.path.abspath(output_folder)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)

    for root, dirs, files in os.walk(fall_dataset_folder):
        print(f"Visiting folder: {root}, Found dirs: {dirs}, Found files: {files}")
        
        # 'FallDataset' 폴더를 건너뛰고 진행
        if os.path.basename(root) == "FallDataset":
            continue

        if 'Annotation_files' in dirs and 'Videos' in dirs:
            annotation_folder = os.path.join(root, 'Annotation_files')
            video_folder = os.path.join(root, 'Videos')
            location_name = os.path.basename(root)
            
            # 장소별 비디오 처리
            process_location_videos(annotation_folder, video_folder, location_name, output_folder)
        else:
            print(f"No Annotation_files or Videos found in {root}")

# 파일 경로 설정
fall_dataset_folder = os.path.abspath('FallDataset')  # 절대 경로로 변환
output_folder = 'output_video_segments'  # 추출된 비디오를 저장할 폴더 경로

# FallDataset 내 모든 비디오와 Annotation 파일 처리
process_fall_dataset(fall_dataset_folder, output_folder)
